In [1]:
import pandas as pd
import os
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
from tqdm.autonotebook import tqdm

//anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
path = os.path.join(".", "data", "data_1908_cleaned.csv")
people_1908_raw = pd.read_csv(path)
people_1908_raw = people_1908_raw.fillna("")
people_1908_raw = people_1908_raw.reset_index()[['Names', 'Addresses']]

In [3]:
path = os.path.join('data', 'data_1884_cleaned.csv')
people_1884_raw = pd.read_csv(path)
people_1884_raw = people_1884_raw[['Names', 'Addresses']]

In [4]:
paris_coord = [48.864716, 2.349014]
paris_map = folium.Map(location=paris_coord)

In [8]:
def get_coord(df):

    notfound_addresses = 0
    people = df.copy()
    people["latitude"] = "0"
    people["longitude"] = "0"
    geolocator = Nominatim(user_agent="agent")

    for i in tqdm(range(len(people))):
        try:
            coord = geolocator.geocode(people['Addresses'][i] + ", Paris")
        except: 
            print("Try again...")
            try:
                coord = geolocator.geocode(people['Addresses'][i] + ", Paris")
            except:
                print("I give up")
        if(coord):
            people["latitude"][i] = coord.latitude
            people["longitude"][i] = coord.longitude
        else: 
            notfound_addresses += 1
    print("Missed " + str(notfound_addresses) + "/" + str(len(people)) + \
          " addresses (= " + str(notfound_addresses/len(people)*100) + "%)")

    return people

def put_people_on_map(people, map_):

    mc = MarkerCluster()


    for i in range(len(people)):
        name = people["Names"][i]
        address = people["Addresses"][i]

        html= """
            <h1> """+ name + """</h1>
            <img src = "https://workingwithdog.com/wp-content/uploads/2016/05/new_instagram_logo-1024x1024.jpg" width = 50 height = 50/>
            <p> Influencer of the past </p>
            <h2> """ + address + """ </h2"""

        iframe = folium.IFrame(html=html, width=400, height=200)
        popup = folium.Popup(iframe, max_width=2650)
        mc.add_child(folium.Marker([people["latitude"][i], people["longitude"][i]], popup=popup))

    map_.add_child(mc)

In [6]:
people_1884 = get_coord(people_1884_raw[:30])
people_1884.head()

Try again...
Try again...
Try again...
Try again...
Try again...

Missed 6/30 addresses (= 20.0%)


,Names,Addresses,latitude,longitude
0,M° Adam.,Avenue Champs-Elysées 53,48.8712,2.3039
1,M. Adam Alfred.,Rue Monceau 67,48.8796,2.30816
2,M° Adam Edmond .,Boulevard Poissonnière 23,48.8711,2.34417
3,Mlle Addenet.,Rue Blairault 19,0,0
4,Bno d’ Adelsward.,"Rue De La Bienfaisance,44",48.8766,2.31487


In [7]:
people_1908 = get_coord(people_1908_raw[:30])
people_1908.head()

Try again...
Try again...
Try again...
Try again...
Try again...
Try again...

Missed 7/30 addresses (= 23.333333333333332%)


,Names,Addresses,latitude,longitude
0,Michel-Robert ABRADIE-d’ARRAST,5S bis rue Jouffroy,0,0
1,ABBATUCCI,103 rue La Boétie,48.8713,2.30717
2,ABBATUCCI,32 rue Washington,48.8736,2.30377
3,Georges ABOILARI,46 avenue de Breteuil,48.8501,2.31147
4,Christian ABOVILLE et Mme née IIENNE-CART,23 rue de Grenelle,48.8534,2.32823


In [9]:
put_people_on_map(people_1884, paris_map)
paris_map